# PyBullet demo

This notebook demonstrates how to train a USA model on a PyBullet environment. The environment is taken from [here](https://github.com/adubredu/pybullet_kitchen).

To get started, first make sure to install some dependencies:

```bash
pip install pybullet scikit-learn scipy
```

In [ ]:
import pybullet as pb
import numpy as np
import imageio
from typing import Iterator
from torch import Tensor
from pathlib import Path
import zipfile
import requests
from IPython.display import Image

The code below downloads the environment data and adds it to PyBullet.

In [ ]:
# Downloads the dataset, if it is not already downloaded.
if not Path("04_pybullet_data").exists():
    r = requests.get("https://github.com/codekansas/usa/releases/download/v0.0.2/04_pybullet_data.zip", allow_redirects=True)
    with open("04_pybullet_data.zip", "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile("04_pybullet_data.zip", "r") as zip_ref:
        zip_ref.extractall(".")

# Loads the URDFs into PyBullet.
pb.setAdditionalSearchPath("04_pybullet_data")
kitchen_path = "kitchen_part_right_gen_convex.urdf"
use_fixed_base = True
pb.setGravity(0, 0, -9.81)

floor = pb.loadURDF(
    "floor.urdf",
    useFixedBase=use_fixed_base,
)

kitchen = pb.loadURDF(
    "kitchen_part_right_gen_convex.urdf",
    (-5, 0, 1.477),
    useFixedBase=use_fixed_base,
)

table = pb.loadURDF(
    "table.urdf",
    (1.0, 0, 0),
    pb.getQuaternionFromEuler((0, 0, 1.57)),
    useFixedBase=use_fixed_base,
)

In [ ]:
def capture_frame(
    camera_xyz: tuple[float, float, float] = (-5.0, 0.0, 1.477),
    camera_ypr: tuple[float, float] = (90.0, -10.0, 0.0),
    camera_distance: float = 3.0,
    camera_planes: tuple[float, float] = (0.01, 100.0),
    pixel_dims: tuple[int, int] = (500, 300),
    camera_fov: float = 60.0,
) -> tuple[np.ndarray, np.ndarray]:
    """Captures a single frame, returning RGB and depth information.

    Args:
        camera_xyz: The XYZ coordinates of the camera
        camera_ypr: The yaw, pitch and roll of the camera
        cam_distance: Not sure
        camera_planes: The minimum and maximum rendering distances
        pixel_dims: The shape of the output image, as (W, H)
        camera_fov: The camera field of view
    """

    yaw, pitch, roll = camera_ypr
    near_plane, far_plane = camera_planes
    pixel_width, pixel_height = pixel_dims

    # Computes the view and projection matrices.
    view_matrix = pb.computeViewMatrixFromYawPitchRoll(camera_xyz, camera_distance, yaw, pitch, roll, 2)
    aspect = pixel_width / pixel_height
    projection_matrix = pb.computeProjectionMatrixFOV(camera_fov, aspect, near_plane, far_plane)

    print(view_matrix)
    print(projection_matrix)

    # Captures the camera image.
    img_arr = pb.getCameraImage(pixel_width, pixel_height, view_matrix, projection_matrix)
    img_width, img_height, rgb, depth, *_ = img_arr

    # Reshapes arrays to expected output shape.
    rgb_arr = np.reshape(rgb, (img_height, img_width, 4))[..., :3]
    depth_arr = np.reshape(depth, (img_height, img_width))

    return rgb_arr, depth_arr


def capture_sim(steps_per_frame: int, total_steps: int) -> Iterator[tuple[np.ndarray, np.ndarray]]:
    for i in range(total_steps):
        pb.stepSimulation()
        if i % steps_per_frame == 0:
            yield capture_frame()


def write_gif(frames: Iterator[np.ndarray | Tensor], out_file: str | Path, *, fps: int = 30) -> None:
    writer = imageio.get_writer(str(out_file), mode="I", fps=fps)
    for frame in frames:
        if isinstance(frame, Tensor):
            frame = frame.detach().cpu().numpy()
        writer.append_data(frame)
    writer.close()

In [ ]:
class Kitchen:
    def __init__(self):
        pb.setAdditionalSearchPath("04_pybullet_data")
        kitchen_path = "kitchen_part_right_gen_convex.urdf"
        useFixedBase = True
        pb.setGravity(0, 0, -9.81)
        self.floor = pb.loadURDF("floor.urdf", useFixedBase=useFixedBase)
        self.kitchen = pb.loadURDF(kitchen_path, [-5, 0, 1.477], useFixedBase=useFixedBase)
        self.table = pb.loadURDF(
            "table.urdf",
            [1.0, 0, 0],
            pb.getQuaternionFromEuler([0, 0, 1.57]),
            useFixedBase=useFixedBase,
        )

In [ ]:
pb.connect(pb.DIRECT)
pb.resetSimulation()

pb.setGravity(0, 0, -9.81)
pb.setPhysicsEngineParameter(enableConeFriction=0)

total_steps_per_control = 1000
steps_per_frame = 100

kitchen = Kitchen()

def iter_frames() -> Iterator[np.ndarray]:
    for i in range(10):
        drawer_id = i + 1
        kitchen.open_drawer(drawer_id)
        yield from capture_sim(steps_per_frame, total_steps_per_control)

    for i in range(10):
        drawer_id = i + 1
        kitchen.close_drawer(drawer_id)
        yield from capture_sim(steps_per_frame, total_steps_per_control)

write_gif(iter_frames(), "video.gif")
Image("video.gif")